In [9]:
import sys
import torch
from torch import nn
import os
import numpy as np

sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1

In [10]:
from csv_data import WeatherDatasetWrapper

class WeatherDataset(WeatherDatasetWrapper):
    predictors = ["tmax", "tmin", "rain"]
    target = "tmax_tomorrow"
    sequence_length = 7

wrapper = WeatherDataset()
datasets = wrapper.generate_datasets(BATCH_SIZE)

In [11]:
train = datasets["train"]
valid = datasets["validation"]
test = datasets["test"]

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.rnn = nn.RNN(3, 4, 1, batch_first=True)
        self.dense = nn.Linear(4, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [20]:
def mse(actual, predicted):
    return torch.mean((actual-predicted)**2)

In [22]:
model = NeuralNetwork().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

EPOCHS = 50
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(DEVICE)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

        epoch_loss += mse(target, pred)

    losses = []
    with torch.no_grad():
        valid_loss = 0
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(DEVICE)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            valid_loss += mse(target, pred)

    print(f"Epoch {epoch} train loss: {epoch_loss / len(train)} valid loss: {valid_loss / len(valid)}")

Epoch 0 train loss: 314.6138000488281 valid loss: 31.349842071533203
Epoch 1 train loss: 25.994112014770508 valid loss: 22.94846534729004
Epoch 2 train loss: 23.340198516845703 valid loss: 22.29945945739746
Epoch 3 train loss: 22.787185668945312 valid loss: 21.83952522277832
Epoch 4 train loss: 22.60393714904785 valid loss: 21.455257415771484
Epoch 5 train loss: 22.515172958374023 valid loss: 21.09100914001465
Epoch 6 train loss: 22.4509220123291 valid loss: 20.907573699951172
Epoch 7 train loss: 22.3182430267334 valid loss: 21.142229080200195
Epoch 8 train loss: 22.219058990478516 valid loss: 21.186647415161133
Epoch 9 train loss: 22.14150047302246 valid loss: 21.70040512084961
Epoch 10 train loss: 22.10215950012207 valid loss: 20.82554054260254
Epoch 11 train loss: 22.07602310180664 valid loss: 20.851511001586914
Epoch 12 train loss: 22.043928146362305 valid loss: 20.940540313720703
Epoch 13 train loss: 22.03813934326172 valid loss: 21.482694625854492
Epoch 14 train loss: 21.97466659